In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [3]:
with open('/Users/JoshuaRohn/.secret/yelp_api.json')as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [5]:
LOCATION = 'NY,NY'
TERM = 'Pizza'

In [6]:
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE

'Data/results_in_progress_NY_pizza.json'

In [7]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


In [8]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
total_results = results['total']
total_results

12500

In [11]:
results_per_page = len(results['businesses'])
results_per_page

20

In [12]:
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

625

In [13]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [14]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/625 [00:00<?, ?it/s]

In [15]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/625 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [16]:
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [17]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [18]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


615

In [19]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/615 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [20]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/od36nF...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2693,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,308.569844
1,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,5031,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[delivery, pickup]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1961.877142
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3153,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
3,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/KKB2Tr...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,1090,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,3145.016041
4,WIhm0W9197f_rRtDziq5qQ,lombardis-pizza-new-york-4,Lombardi's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/lBq1IB...,False,https://www.yelp.com/biz/lombardis-pizza-new-y...,6608,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7215934960083, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '32 Spring St', 'address2': '', '...",+12129417994,(212) 941-7994,1798.995978


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,FER-iaOyA8JFH-DL5vaKIQ,vesuvio-brooklyn-2,Vesuvio,https://s3-media3.fl.yelpcdn.com/bphoto/pBEF-I...,False,https://www.yelp.com/biz/vesuvio-brooklyn-2?ad...,448,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 40.632565, 'longitude': -74.027047}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '7305 3rd Ave', 'address2': '', '...",+17187450222,(718) 745-0222,8561.516037
996,2Nwmd7-1d1rI6lb9QSSYYA,a-pasta-bar-new-york,A Pasta Bar,https://s3-media2.fl.yelpcdn.com/bphoto/4VDTeJ...,False,https://www.yelp.com/biz/a-pasta-bar-new-york?...,524,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.72229, 'longitude': -74.00403}","[pickup, delivery]",$$$,"{'address1': '330 W Broadway', 'address2': '',...",+16467624056,(646) 762-4056,2048.470505
997,D4vcEbaq1DjFG6FVk26ReQ,la-amistad-pizzeria-and-grill-new-york,La Amistad Pizzeria & Grill,https://s3-media2.fl.yelpcdn.com/bphoto/0gUNvG...,False,https://www.yelp.com/biz/la-amistad-pizzeria-a...,28,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",4.5,"{'latitude': 40.79098, 'longitude': -73.94241}","[pickup, delivery]",$,"{'address1': '2067 2nd Ave', 'address2': None,...",+12122560129,(212) 256-0129,10471.666962
998,HoIQaDaSkDDCH7I6plia4A,the-pacific-street-pizza-greenvale-3,The Pacific Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/2TEA65...,False,https://www.yelp.com/biz/the-pacific-street-pi...,16,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.811116, 'longitude': -73.628024}",[],NaN,"{'address1': '38 Glen Cove Rd', 'address2': ''...",+15162311950,(516) 231-1950,33013.972080
999,qgbyJnZzLXnFsNqowFc1_g,fresh-pizza-ridgewood-2,Fresh Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/vHV029...,False,https://www.yelp.com/biz/fresh-pizza-ridgewood...,4,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 40.7030459, 'longitude': -73.895523}","[pickup, delivery]",NaN,"{'address1': '69-08 Fresh Pond Rd', 'address2'...",+13475991447,(347) 599-1447,8332.430145


In [21]:
final_df.duplicated().sum()

TypeError: unhashable type: 'list'

In [22]:
final_df.duplicated(subset='id').sum()

121

In [23]:
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [24]:
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)